In [13]:
# DATA PREPARATION
# --------------------------------------------------------------------------------------

# Select data: Determine which data sets will be used and document reasons for 
# inclusion/exclusion.
# 
# Clean data: Often this is the lengthiest task. Without it, you’ll likely fall victim
# to garbage-in, garbage-out. A common practice during this task is to correct, impute,
# or remove erroneous values.
# 
# Construct data: Derive new attributes that will be helpful. For example, derive 
# someone’s body mass index from height and weight fields.
# 
# Integrate data: Create new data sets by combining data from multiple sources.
# 
# Format data: Re-format data as necessary. For example, you might convert string values
# that store numbers to numeric values so that you can perform mathematical operations.

In [14]:
import os

import pandas as pd
from ydata_profiling import ProfileReport

pd.set_option("display.max_columns", None)
YEAR = "2021"

os.makedirs(f"../output/c-data-preparation", exist_ok=True)
os.makedirs(f"../data/c-data-preparation", exist_ok=True)

In [15]:
# Read data

file = f"../data/b-data-understanding/{YEAR}-grouped-by-occurrence.csv"

df = pd.read_csv(file)

described = df.describe(include="all")
described.loc["nan"] = df.isnull().mean() * 100
described

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
count,19770.000000,19770,19770,19770,19770,19727.000000,19727.000000,19770,19770,19770,19770,19770,19770,19770,19770,19770,19770,19770.000000,19770.000000,19770.000000,19770.000000,19770.000000,19770.000000,19770.000000,19770.000000,19770.000000,19770.000000,19770,19770,19669
unique,NaN,365,7,903,3,NaN,NaN,445,69,17,3,4,3,10,3,10,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,29,93
top,NaN,2021-08-08,sábado,19:00:00,SC,NaN,NaN,CURITIBA,Velocidade Incompatível,Colisão traseira,Com Vítimas Feridas,Pleno dia,Crescente,Céu Claro,Simples,Reta,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPRF-SC,DEL01-PR,UOP01-DEL01-SC
freq,NaN,101,3403,254,7890,NaN,NaN,900,2329,3888,14922,10610,10305,10885,9455,11398,10363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7918,3032,1035
mean,369542.644461,NaN,NaN,NaN,NaN,256.400568,221.504648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.291047,0.062873,0.837683,0.246232,1.006070,0.138189,1.083915,1.686141,-26.936908,-50.609010,NaN,NaN,NaN
std,21759.173611,NaN,NaN,NaN,NaN,133.527176,179.519283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.639392,0.310085,0.944891,0.565921,1.251622,0.399331,1.047696,0.719060,1.969246,1.856358,NaN,NaN,NaN
min,331693.000000,NaN,NaN,NaN,NaN,101.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-33.689819,-57.502806,NaN,NaN,NaN
25%,350965.750000,NaN,NaN,NaN,NaN,116.000000,92.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,-28.196260,-51.914951,NaN,NaN,NaN
50%,369583.000000,NaN,NaN,NaN,NaN,277.000000,173.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,2.000000,-26.880509,-50.158686,NaN,NaN,NaN
75%,388376.750000,NaN,NaN,NaN,NaN,376.000000,313.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,2.000000,-25.502576,-48.956708,NaN,NaN,NaN


In [16]:
# Select features
# --------------------------------------------------------------------------------------

# Only some features will be used for clustering. However, other features are also
# interesting for further analysis, when the clusters are known. Therefore, we will keep
# all of them in the dataset.

In [17]:
# Gather additional data
# --------------------------------------------------------------------------------------

In [18]:
# Clean data
# --------------------------------------------------------------------------------------

# 1. Drop nans

n1 = len(df)
df = df.dropna()
n2 = len(df)

df = df.reset_index(drop=True)

# TODO: impute missing values
# 
# Only 3 columns with missing values: br, km, uop
# They can be imputed by considering lat, long, and city features

print(f"Number of rows dropped: {n1 - n2}")

# 2. Drop useless columns

df = df.drop(columns=["id"])

# 3. Format text data

# TODO: don't think it's necessary, text looks consistent (e.g. no typos)
# 
# str: lower case
# str: remove punctuation
# str: remove special characters
# str: remove accents
# str: remove extra spaces

Number of rows dropped: 139


In [19]:
# Feature engineering
# --------------------------------------------------------------------------------------

df.data_inversa = pd.to_datetime(df.data_inversa, format="%Y-%m-%d").dt.date
df.horario = pd.to_datetime(df.horario, format="%H:%M:%S").dt.time

func = lambda row: pd.Timestamp.combine(row.data_inversa, row.horario)
df["timestamp"] = df.apply(func, axis=1)

df["ano"] = df.timestamp.dt.year
df["mes"] = df.timestamp.dt.month
df["dia"] = df.timestamp.dt.day
# Already have "dia_semana" column
# df["dia_semana"] = df.data_inversa.dt.dayofweek

df["hora"] = df.timestamp.dt.hour
df["minuto"] = df.timestamp.dt.minute

df = df.drop(columns=["data_inversa", "horario", "timestamp"])

In [20]:
# Data type conversion
# --------------------------------------------------------------------------------------

dtypes = {
    # "id": "Int64",
    # "data_inversa": "datetime64[ns]",
    "dia_semana": "category",
    # "horario": "datetime64[ns]",
    "uf": "category",
    "br": "category",
    "km": "float64",
    "municipio": "category",
    "causa_acidente": "category",
    "tipo_acidente": "category",
    "classificacao_acidente": "category",
    "fase_dia": "category",
    "sentido_via": "category",
    "condicao_metereologica": "category",
    "tipo_pista": "category",
    "tracado_via": "category",
    "uso_solo": "category",
    "pessoas": "Int64",
    "mortos": "Int64",
    "feridos_leves": "Int64",
    "feridos_graves": "Int64",
    "ilesos": "Int64",
    "ignorados": "Int64",
    "feridos": "Int64",
    "veiculos": "Int64",
    "latitude": "float64",
    "longitude": "float64",
    "regional": "category",
    "delegacia": "category",
    "uop": "category",
    "ano": "Int64",
    "dia": "Int64",
    "mes": "Int64",
    "hora": "Int64",
    "minuto": "Int64",
}

df = df.astype(dtypes)

In [21]:
df.to_csv("../data/c-data-preparation/dataset.csv", index=False)

In [22]:
# Encoding and normalization
# --------------------------------------------------------------------------------------

import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Check if all columns have allowed data types
allowed_dtypes = ["category", "Int64", "int64", "float64"]
msg = "Invalid data type. Allowed data types: category, Int64, float64"
assert all(df[col].dtype.name in allowed_dtypes for col in df.columns), msg

# Encode categorical columns
# --------------------------------------------------------------------------------------
label_encoders = {}

for column in df.select_dtypes(include=["category"]):
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

with open("../output/c-data-preparation/label-encoders.pkl", "wb") as fp:
    pickle.dump(label_encoders, fp)

# Load the label encoders from the pickle file
# with open("../output/c-data-preparation/label-encoders.pkl", "rb") as fp:
#     label_encoders = pickle.load(fp)

# for column in df.select_dtypes(include=["category"]):
#     df[column] = label_encoders[column].inverse_transform(df[column].astype(int))

# Normalize all columns
# --------------------------------------------------------------------------------------

# NOTE: # It's better to use StandardScaler instead of MinMaxScaler for lat and long.
# The reason is that latitude and longitude represent angular distances on the Earth's
# surface, and Min-Max scaling would distort these angular distances.

# scaler = MinMaxScaler()
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

with open("../output/c-data-preparation/min-max-scaler.pkl", "wb") as fp:
    pickle.dump(scaler, fp)

# Load the scaler from the pickle file
# with open("../output/c-data-preparation/min-max-scaler.pkl", "rb") as fp:
#     scaler = pickle.load(fp)

df.head()

,dia_semana,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,ano,mes,dia,hora,minuto
0,0.504431,-1.179492,-0.281066,-0.945450,1.328328,0.723221,1.301798,-0.287018,0.776219,1.046786,-0.263606,-1.115796,-1.879257,-0.953745,1.041751,-0.202223,0.170337,-0.433989,1.593202,-0.345792,-0.080292,1.827446,0.686189,0.914287,-1.183359,-1.231548,1.788331,0.0,-1.637935,-1.660056,0.313374,1.139094
1,0.504431,1.102312,1.705964,-0.790699,-0.222842,1.334716,-1.102036,-2.390991,0.776219,-0.955305,1.464914,0.998406,0.598758,-0.953745,4.087807,6.242317,4.403097,1.336876,-0.802581,2.157246,4.693209,1.827446,-0.002548,0.699267,1.099290,0.149601,-0.985191,0.0,-1.637935,-1.660056,0.634235,-0.869757
2,0.504431,1.102312,1.705964,-0.507360,0.064412,-1.460689,-0.227915,-0.287018,-0.516777,1.046786,1.464914,0.998406,-0.392448,1.048498,1.041751,-0.202223,-0.887853,3.107741,0.794608,-0.345792,0.874408,0.436581,-0.108575,0.564696,1.099290,0.149601,0.073127,0.0,-1.637935,-1.660056,0.955095,1.426073
3,0.504431,-1.179492,-0.902013,-0.902587,-1.158468,-0.281378,-1.757627,-2.390991,-0.516777,-0.955305,-0.955013,-1.115796,-0.392448,1.048498,-0.176672,3.020047,-0.887853,-0.433989,-0.003987,-0.345792,-1.034992,-0.954285,0.834357,0.894492,-1.183359,-1.231548,-0.328304,0.0,-1.637935,-1.660056,1.115525,0.852116
4,1.004910,1.102312,-1.026202,-0.607558,-0.148977,1.247360,-0.009384,-0.287018,0.776219,1.046786,1.810617,-1.115796,-0.392448,-0.953745,1.041751,-0.202223,0.170337,1.336876,0.794608,-0.345792,0.874408,1.827446,0.051110,1.014225,1.099290,0.149601,1.532875,0.0,-1.637935,-1.546410,-0.649206,-0.008821


In [23]:
described = df.describe(include="all")
described.loc["nan"] = df.isnull().mean() * 100
described

,dia_semana,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,ano,mes,dia,hora,minuto
count,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,19631.000000,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04,19631.0,1.963100e+04,1.963100e+04,1.963100e+04,1.963100e+04
mean,7.962885e-17,6.623673e-17,-4.741536e-17,-5.791189e-17,9.193513e-17,0.000000,4.832024e-17,-9.917412e-17,-1.447797e-17,2.099306e-17,-7.311377e-17,4.198612e-17,-6.515088e-17,5.791189e-17,7.238987e-17,7.238987e-17,5.212070e-17,-5.067291e-17,-4.343392e-17,-1.447797e-18,3.329934e-17,5.212070e-17,1.690303e-15,2.823205e-16,-2.967985e-17,-1.737357e-17,3.329934e-17,0.0,4.632951e-17,5.212070e-17,-3.329934e-17,-4.198612e-17
std,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,0.0,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00
min,-1.497485e+00,-1.179492e+00,-1.026202e+00,-1.231015e+00,-1.708354e+00,-1.460689,-1.976158e+00,-2.390991e+00,-3.102769e+00,-9.553048e-01,-9.550132e-01,-1.115796e+00,-1.879257e+00,-9.537454e-01,-7.858828e-01,-2.022228e-01,-8.878534e-01,-4.339886e-01,-8.025810e-01,-3.457918e-01,-1.034992e+00,-9.542851e-01,-3.424008e+00,-3.711855e+00,-1.183359e+00,-1.231548e+00,-1.386621e+00,0.0,-1.637935e+00,-1.660056e+00,-2.093078e+00,-1.443715e+00
25%,-9.970063e-01,-1.179492e+00,-9.020131e-01,-7.194466e-01,-8.383855e-01,-0.892873,-6.649754e-01,-2.870175e-01,-5.167769e-01,-9.553048e-01,-6.093093e-01,-1.115796e+00,-3.924480e-01,-9.537454e-01,-7.858828e-01,-2.022228e-01,-8.878534e-01,-4.339886e-01,-8.025810e-01,-3.457918e-01,-8.029189e-02,-9.542851e-01,-6.409515e-01,-7.065267e-01,-1.183359e+00,-9.804298e-01,-9.486969e-01,0.0,-7.699710e-01,-8.645367e-01,-8.096366e-01,-8.697574e-01
50%,3.951619e-03,-3.858993e-02,-2.810662e-01,-2.735639e-01,-3.407527e-02,0.242761,-9.384194e-03,-2.870175e-01,7.762193e-01,-9.553048e-01,-6.093093e-01,-5.869493e-02,5.987578e-01,-9.537454e-01,-1.766716e-01,-2.022228e-01,1.703368e-01,-4.339886e-01,-3.986666e-03,-3.457918e-01,-8.029189e-02,4.365807e-01,3.229355e-02,2.543510e-01,-4.203442e-02,-1.015166e-01,-1.458354e-01,0.0,9.799282e-02,-6.901751e-02,1.529443e-01,1.059706e-01
75%,1.004910e+00,1.102312e+00,8.366383e-01,5.088184e-01,8.441001e-01,0.941613,1.301798e+00,-2.870175e-01,7.762193e-01,1.046786e+00,1.464914e+00,9.984061e-01,5.987578e-01,1.048498e+00,4.325397e-01,-2.022228e-01,1.703368e-01,-4.339886e-01,-3.986666e-03,-3.457918e-01,-8.029189e-02,4.365807e-01,7.297472e-01,8.900444e-01,1.099290e+00,9.029556e-01,9.124820e-01,0.0,9.659566e-01,8.401472e-01,7.946650e-01,8.521156e-01
max,1.505389e+00,1.102312e+00,2.451100e+00,2.838152e+00,1.894628e+00,1.509429,1.520329e+00,1.816956e+00,7.762193e-01,1.046786e+00,2.156321e+00,9.984061e-01,2.581169e+00,1.048498e+00,3.150231e+01,6.102091e+01,3.509061e+01,2.612898e+01,3.593276e+01,1.467244e+01,3.333422e+01,1.434524e+01,2.117075e+00,1.128220e+00,1.099290e+00,2.284105e+00,1.934306e+00,0.0,1.544599e+00,1.749312e+00,1.596816e+00,1.942635e+00
nan,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00

In [24]:
df.to_csv("../data/c-data-preparation/prepared.csv", index=False)

In [25]:
# Profile report

# profile = ProfileReport(df, title="Profiling Report")
# profile.to_file("../output/c-data-preparation/profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 